In [ ]:
# !pip install git+https://github.com/huggingface/transformers.git
!pip install seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.7.10


In [ ]:
!pip install transformers==3.0.1

     |████████████████████████████████| 757kB 8.5MB/s 
     |████████████████████████████████| 3.0MB 44.7MB/s 
     |████████████████████████████████| 901kB 44.9MB/s 
     |████████████████████████████████| 1.2MB 47.4MB/s 


In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import json
import gc

In [ ]:
print(tf.__version__)

2.5.0


In [ ]:
col_names=['category','B','text']
df = pd.read_csv('/content/drive/MyDrive/Data/Case3_Dataset.csv',names=col_names,header=None)
df.head()

,category,B,text
0,airline_sentiment,airline,text
1,neutral,Virgin America,@VirginAmerica What @dhepburn said.
2,positive,Virgin America,@VirginAmerica plus you've added commercials t...
3,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
4,negative,Virgin America,@VirginAmerica it's really aggressive to blast...


In [ ]:
df=df.drop('B',axis=1)
df = df.iloc[1: , :]

In [ ]:
df['encoded_cat'] = df['category'].astype('category').cat.codes
df.head()

,category,text,encoded_cat
1,neutral,@VirginAmerica What @dhepburn said.,1
2,positive,@VirginAmerica plus you've added commercials t...,2
3,neutral,@VirginAmerica I didn't today... Must mean I n...,1
4,negative,@VirginAmerica it's really aggressive to blast...,0
5,negative,@VirginAmerica and it's a really big bad thing...,0


In [ ]:
df['category'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: category, dtype: int64

In [ ]:
df.tail()

,category,text,encoded_cat
14636,positive,@AmericanAir thank you we got on a different f...,2
14637,negative,@AmericanAir leaving over 20 minutes Late Flig...,0
14638,neutral,@AmericanAir Please bring American Airlines to...,1
14639,negative,"@AmericanAir you have my money, you change my ...",0
14640,neutral,@AmericanAir we have 8 ppl so we need 2 know h...,1


In [ ]:
data_texts = df["text"].to_list() 
data_labels = df["encoded_cat"].to_list()

In [ ]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=0)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Streaming output truncated to the last 5000 lines.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' trun

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=151)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_39', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=5, batch_size=16, validation_data=val_dataset.shuffle(1000).batch(16))

Epoch 1/5
725/725 [==============================] - 73s 92ms/step - loss: 0.6554 - accuracy: 0.7891 - val_loss: 0.4368 - val_accuracy: 0.8224
Epoch 2/5
725/725 [==============================] - 65s 90ms/step - loss: 0.3224 - accuracy: 0.8836 - val_loss: 0.4730 - val_accuracy: 0.8204
Epoch 3/5
725/725 [==============================] - 65s 90ms/step - loss: 0.1909 - accuracy: 0.9357 - val_loss: 0.6473 - val_accuracy: 0.8186
Epoch 4/5
725/725 [==============================] - 65s 90ms/step - loss: 0.1214 - accuracy: 0.9615 - val_loss: 0.6256 - val_accuracy: 0.8268
Epoch 5/5
725/725 [==============================] - 66s 91ms/step - loss: 0.0840 - accuracy: 0.9739 - val_loss: 0.7702 - val_accuracy: 0.8180


In [ ]:
save_directory = "/content/drive/MyDrive/Data" 

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/Data/vocab.txt',
 '/content/drive/MyDrive/Data/special_tokens_map.json',
 '/content/drive/MyDrive/Data/added_tokens.json')

In [ ]:
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some weights of the model checkpoint at /content/drive/MyDrive/Data were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Data and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_text="@VirginAmerica plus you've added commercials to the experience... tacky."
test_text

"@VirginAmerica plus you've added commercials to the experience... tacky."

In [ ]:
predict_input = loaded_tokenizer.encode(test_text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

output = loaded_model(predict_input)[0]

prediction_value = tf.argmax(output, axis=1).numpy()[0]
df.loc[df['encoded_cat'] == prediction_value, 'category'].iloc[0]

'positive'